In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import src.data_extraction.access_database as db
import src.data_extraction.filter_adult_patients as fap
import src.data_extraction.filter_ventilation_events as fve
from src.utils import utils
import psycopg2

# 데이터 저장위치
output_dir = './data'

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

# 호흡기 ITEM ID 설정 (웬만하면 건들지 마세요)
vent_ids_config = {
    'INTUBATION_ITEM_IDS': "224385",
    'EXTUBATION_ITEM_IDS': "225468, 225477, 227194"
}

# 테이블 쿼리 설정
tables_query = {
    'pg_tables': 'SELECT * FROM PG_TABLES;',
    'patients': 'SELECT * FROM mimiciv_hosp.patients;',
    'admissions': 'SELECT * FROM mimiciv_hosp.admissions;',
    'transfers': 'SELECT * FROM mimiciv_hosp.transfers;',
    'icustays': 'SELECT * FROM mimiciv_icu.icustays;',
    'd_items': 'SELECT * FROM mimiciv_icu.d_items;',
    'intubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['INTUBATION_ITEM_IDS']});",
    'extubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['EXTUBATION_ITEM_IDS']});",
    'ventilation': 'SELECT * FROM mimiciv_derived.ventilation;'
}

db.print_config_info(db_config, tables_query)

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

try:
    # 데이터베이스 연결
    conn = psycopg2.connect(**db_config)
    print("Connected to the database successfully!")

    # 커서 생성
    curs = conn.cursor()

    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")

    # SQL 쿼리 실행
    curs.execute("SELECT version();")

    # 결과 가져오기
    result = curs.fetchone()
    print("PostgreSQL version:", result)

except (psycopg2.Error, Exception) as error:
    print("Error while connecting to the database:", error)



C:\Users\inhae\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
INFO:root:Retrieved pg_tables: (164, 8)


--------- Database Configuration ---------
Database Name: mimiciv
User: mai_onlyselect
Password: *************
Host: 1.212.63.162
Port: 35430

--------- SQL Queries for Required Tables ---------
pg_tables: SELECT * FROM PG_TABLES;
patients: SELECT * FROM mimiciv_hosp.patients;
admissions: SELECT * FROM mimiciv_hosp.admissions;
transfers: SELECT * FROM mimiciv_hosp.transfers;
icustays: SELECT * FROM mimiciv_icu.icustays;
d_items: SELECT * FROM mimiciv_icu.d_items;
intubation: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (224385);
extubation: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (225468, 225477, 227194);
ventilation: SELECT * FROM mimiciv_derived.ventilation;

Connected to the database successfully!


INFO:root:Retrieved patients: (299712, 6)
INFO:root:Retrieved admissions: (431231, 16)
INFO:root:Retrieved transfers: (1890972, 7)
INFO:root:Retrieved icustays: (73181, 8)
INFO:root:Retrieved d_items: (4014, 9)
INFO:root:Retrieved intubation: (8488, 22)
INFO:root:Retrieved extubation: (23122, 22)
INFO:root:Retrieved ventilation: (109200, 4)


PostgreSQL version: ('PostgreSQL 15.7 (Ubuntu 15.7-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


# data 불러오기

## lab_abga

In [4]:
curs.execute("select * from mimiciv_hosp.labevents where itemid in (50818, 50820, 50821, 50802, 50817, 50882);")
columns_name = [desc[0] for desc in curs.description]
lab_abga = pd.DataFrame(curs.fetchall(), columns=columns_name)

## ventilator_setting

In [5]:
curs.execute("select * from mimiciv_derived.ventilator_setting;")
columns_name = [desc[0] for desc in curs.description]
ventilator_setting = pd.DataFrame(curs.fetchall(), columns=columns_name)

## ventilation

In [6]:
curs.execute("select * from mimiciv_derived.ventilation;")
columns_name = [desc[0] for desc in curs.description]
ventilation = pd.DataFrame(curs.fetchall(), columns=columns_name)

## gcs

In [10]:
curs.execute("select * from mimiciv_derived.gcs;")
columns_name = [desc[0] for desc in curs.description]
gcs = pd.DataFrame(curs.fetchall(), columns=columns_name)

## subjectlist2

In [8]:
subjectlist2 = pd.read_csv('./data/paired_df.csv'\
                           , parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])

# sub2list
sub2list = subjectlist2 >> select('subject_id','hadm_id','stay_id')

ValueError: Missing column provided to 'parse_dates': 'reintubation_eventtime'

## vitalsign

In [12]:
curs.execute("select * from mimiciv_derived.vitalsign;")
columns_name = [desc[0] for desc in curs.description]
vitalsign = pd.DataFrame(curs.fetchall(), columns=columns_name)

## icystays

In [13]:
curs.execute("select * from mimiciv_icu.icustays;")
columns_name = [desc[0] for desc in curs.description]
icustays = pd.DataFrame(curs.fetchall(), columns=columns_name)
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

# 데이터 전처리

## vitalsign

In [7]:
sub1_vitals = sub2list >> left_join(vitalsign, by=['subject_id','stay_id'])

def calculate_mbp(row):
    if pd.isna(row['mbp']) and pd.notna(row['sbp']) and pd.notna(row['dbp']):
        return (row['sbp'] + 2 * row['dbp']) / 3
    return row['mbp']

# sbp 값을 공식으로 채우는 함수 정의
def calculate_sbp(row):
    if pd.isna(row['sbp']) and pd.notna(row['mbp']) and pd.notna(row['dbp']):
        return 3 * row['mbp'] - 2 * row['dbp']
    return row['sbp']

# dbp 값을 공식으로 채우는 함수 정의
def calculate_dbp(row):
    if pd.isna(row['dbp']) and pd.notna(row['mbp']) and pd.notna(row['sbp']):
        return (3 * row['mbp'] - row['sbp']) / 2
    return row['dbp']

# apply 함수를 사용하여 mbp, sbp, dbp 값을 계산
sub1_vitals['mbp'] = sub1_vitals.apply(calculate_mbp, axis=1)
sub1_vitals['sbp'] = sub1_vitals.apply(calculate_sbp, axis=1)
sub1_vitals['dbp'] = sub1_vitals.apply(calculate_dbp, axis=1)


NameError: name 'sub1list' is not defined

## ventset

In [ ]:
sub1_ventset = sub2list >> left_join(ventilator_setting, by=['subject_id','stay_id'])

## gcs, severity_prepare

In [ ]:
sub2list = subjectlist2 >> select('subject_id','hadm_id','stay_id')
sub1_gcs = sub2list >> left_join(gcs, by=['subject_id','stay_id'])
sub1_gcs_notna = sub1_gcs[sub1_gcs.gcs.notna()]

# motor와 eyes만 가지고 총합 재설정하기
sub1_gcs_notna['gcs_m_e'] = sub1_gcs_notna['gcs_motor'] + sub1_gcs_notna['gcs_eyes']

## lab

In [ ]:
sub1_abga = sub2list >> left_join(lab_abga, by=['subject_id','hadm_id'])

sub1_abga['charttime'] = pd.to_datetime(sub1_abga['charttime'])
sub1_abga = sub1_abga.sort_values(by=['subject_id', 'hadm_id', 'charttime'])

# 전체 중복 제거
sub1_abga_unique_tmp = sub1_abga.drop_duplicates()

# stay_id를 제외한 모든 열을 그룹화
group_columns = sub1_abga_unique_tmp.columns.difference(['stay_id']).tolist()

# 그룹화하여 중복된 데이터 찾기(stay_id만 다르고 모든 데이터 동일)
sub1_abga_unique_tmp.loc[:, 'is_duplicated'] = sub1_abga_unique_tmp.duplicated(subset=group_columns, keep=False)

# 중복된 데이터만 필터링
duplicated_groups = sub1_abga_unique_tmp[sub1_abga_unique_tmp['is_duplicated']]

# 데이터 병합
merged_df = pd.merge(duplicated_groups, icustays, on=['subject_id', 'hadm_id', 'stay_id'], how='inner')

# datetime 형식 변환
merged_df['charttime'] = pd.to_datetime(merged_df['charttime'])
merged_df['intime'] = pd.to_datetime(merged_df['intime'])
merged_df['outtime'] = pd.to_datetime(merged_df['outtime'])

# charttime이 intime과 outtime 사이에 있는 행만 필터링
filtered_df = merged_df[(merged_df['charttime'] >= merged_df['intime']) & (merged_df['charttime'] <= merged_df['outtime'])]

# icustays테이블의 컬럼들 삭제
columns_to_drop = ['first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'is_duplicated']
final_df = filtered_df.drop(columns=columns_to_drop)

# is_duplicated 컬럼이 True인 것을 모두 삭제하고, final_df와 병합
sub1_abga_unique = pd.concat([final_df, sub1_abga_unique_tmp[~sub1_abga_unique_tmp.index.isin(duplicated_groups.index)]])

# 다시 정렬
sub1_abga_unique = sub1_abga_unique.sort_values(by=['subject_id', 'hadm_id', 'charttime'])
sub1_abga_unique['charttime'] = pd.to_datetime(sub1_abga_unique['charttime'])
sub1_abga_unique['storetime'] = pd.to_datetime(sub1_abga_unique['storetime'])

# 데이터 저장

In [ ]:
# 경로 맞게 수정
sub1_vitals.to_csv('./data/sub1_vitals.csv', index=False, encoding='utf-8-sig')
sub1_gcs_notna.to_csv('./data/sub1_gcs_notna.csv', index=False, encoding='utf-8-sig')
sub1_ventset.to_csv('./data/sub1_ventset.csv', index=False, encoding='utf-8-sig')
sub1_abga_unique.to_csv('./data/sub1_abga_unique.csv', index=False, encoding='utf-8-sig')
icustays.to_csv('./data/icustays.csv', index=False, encoding='utf-8-sig')